In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

os.chdir("..")

In [2]:
import numpy as np
import yaml
from src.datasets.datasets import CPDDatasets
from src.ensembles.ensembles import EnsembleCPDModel
from src.metrics.evaluation_pipelines import (
    all_cusums_evaluation_pipeline,
    all_distances_evaluation_pipeline,
    evaluate_all_models_in_ensemble,
    evaluation_pipeline,
)
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload
%matplotlib inline

comet_ml is installed but `COMET_API_KEY` is not set.


# Synthetic 1D

In [3]:
model_type = "seq2seq"

experiments_name = "synthetic_1D"

# read config file
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

# prepare datasets
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [4]:
# load the models
path_to_models_folder = "saved_models/bce/synthetic_1D/full_sample"

ens_bce = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_bce.load_models_list(path_to_models_folder)

## Metrics for single models

In [14]:
_ = evaluate_all_models_in_ensemble(
    ens_bce,
    test_dataloader,
    threshold_number=100,
    margin_list=[5, 10, 20],
    verbose=True,
)

100%|██████████| 10/10 [02:19<00:00, 13.97s/it]

AUC: 656.9378 \pm 27.4826
Time to FA: 94.4373 \pm 0.6117
Delay detection: 1.4703 \pm 0.7652
Max F1: 0.9761 \pm 0.0225
Cover: 0.9859 \pm 0.0035
Max cover: 0.9864 \pm 0.0036
Max F1 with m1: 0.9655 \pm 0.023
Max F1 with m2: 0.9722 \pm 0.0229
Max F1 with m3: 0.9732 \pm 0.0216


## Evaluate Ensemble

In [8]:
threshold_number = 100
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

metrics, (max_th_f1_margins_dict, max_f1_margins_dic), _, _ = evaluation_pipeline(
    ens_bce,
    test_dataloader,
    threshold_list,
    device="cpu",
    model_type="ensemble",
    verbose=True,
    margin_list=[5, 10, 20],
)

 40%|████      | 2/5 [00:00<00:00, 12.38it/s]

Collectting model's outputs


100%|██████████| 5/5 [00:00<00:00, 13.58it/s]


TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.8226481119791667
TN: 0, FP: 300, FN: 0, TP: 0

## Evaluate CUSUMs

In [5]:
res = all_cusums_evaluation_pipeline(
    ens_bce,
    threshold_number=300,
    test_dataloader=test_dataloader,
    margin_list=[5, 10, 20],
    device="cpu",
    verbose=True,
    min_th_quant=0.1,
    max_th_quant=0.9,
)

 20%|██        | 1/5 [00:00<00:00,  9.34it/s]

Collectting model's outputs


  0%|          | 0/300 [00:00<?, ?it/s]

Evaluating CUSUM model with cusum_mode = old and conditional = True
Threshold range: (0.0, 5.586112022399902)


  0%|          | 0/300 [00:00<?, ?it/s]

...

Results for best threshold = 0.0
AUDC: 0.0, Time to FA: 94.4733, DD: 0.87, F1: 0.9839, Cover: 0.9885, Max Cover: 0.9885
Max F1 with margin 5: 0.974
Max F1 with margin 10: 0.9806
Max F1 with margin 20: 0.9806
Evaluating CUSUM model with cusum_mode = new_criteria and conditional = False
Threshold range: (3.4608256816864014, 25021.0703125)


  0%|          | 0/300 [00:00<?, ?it/s]

Results for best threshold = 87.13176042020919
AUDC: 0.0, Time to FA: 94.0633, DD: 0.6733, F1: 0.9873, Cover: 0.9915, Max Cover: 0.9915
Max F1 with margin 5: 0.9808
Max F1 with margin 10: 0.9808
Max F1 with margin 20: 0.9808
Evaluating CUSUM model with cusum_mode = new_criteria and conditional = True
Threshold range: (3.4586973190307617, 1129.7864990234375)


100%|██████████| 300/300 [01:34<00:00,  3.17it/s]

Results for best threshold = 82.56533222134695
AUDC: 0.0, Time to FA: 94.07, DD: 0.7, F1: 0.984, Cover: 0.991, Max Cover: 0.991
Max F1 with margin 5: 0.9775
Max F1 with margin 10: 0.9775
Max F1 with margin 20: 0.9808


## Evaluate distances

In [5]:
res = all_distances_evaluation_pipeline(
    ens_bce,
    test_dataloader,
    distance="wasserstein_1d",
    device="cpu",
    verbose=True,
    window_size_list=[2, 4, 8, 16],
    margin_list=[5, 10, 20],
    anchor_window_type_list=["start", "prev"],
    threshold_list=np.linspace(0, 1, 100),
)

 40%|████      | 2/5 [00:00<00:00, 15.72it/s]

Collectting model's outputs


  0%|          | 0/100 [00:00<?, ?it/s]

window_size = 2, anchor_window_type = start


  0%|          | 0/100 [00:00<?, ?it/s]

...

Results for best threshold = 0.16161616161616163
AUDC: 0.0, Time to FA: 95.5467, DD: 3.1233, F1: 0.974, Cover: 0.9694, Max Cover: 0.9694
Max F1 with margin 5: 0.9396
Max F1 with margin 10: 0.9707
Max F1 with margin 20: 0.9707
window_size = 16, anchor_window_type = start


  0%|          | 0/100 [00:00<?, ?it/s]

Results for best threshold = 0.08080808080808081
AUDC: 0.0, Time to FA: 95.1933, DD: 3.4, F1: 0.9709, Cover: 0.9657, Max Cover: 0.9657
Max F1 with margin 5: 0.9034
Max F1 with margin 10: 0.9539
Max F1 with margin 20: 0.9675
window_size = 16, anchor_window_type = prev


100%|██████████| 100/100 [04:36<00:00,  2.77s/it]

Results for best threshold = 0.08080808080808081
AUDC: 0.0, Time to FA: 94.9933, DD: 3.2333, F1: 0.9742, Cover: 0.967, Max Cover: 0.967
Max F1 with margin 5: 0.9034
Max F1 with margin 10: 0.9539
Max F1 with margin 20: 0.9675


# HAR BCE

In [3]:
model_type = "tscp"

experiments_name = "human_activity"

# read config file
path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

# prepare datasets
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [4]:
ens_num = 1

# load the models
# path_to_models_folder = f"saved_models/{args_config['loss_type']}/{experiments_name}/full_sample/ens_{ens_num}"
path_to_models_folder = f"saved_models/tscp/human_activity/window_{args_config['model']['window']}/ens_{ens_num}"

ens_bce = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_bce.load_models_list(path_to_models_folder)

In [5]:
from sklearn.model_selection import train_test_split
from src.utils.calibration import calibrate_all_models_in_ensemble

In [6]:
_, val_dataset = train_test_split(train_dataset, test_size=0.3, random_state=42)
val_dataloader = DataLoader(
    val_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

_ = calibrate_all_models_in_ensemble(
    ens_bce,
    val_dataloader,
    cal_type="beta",
    verbose=True,
    device="cpu",
)

  1%|▏         | 2/135 [00:00<00:08, 16.08it/s]

Collectting model's outputs


  1%|          | 1/135 [00:00<00:18,  7.35it/s]

Collectting model's outputs


  1%|          | 1/135 [00:00<00:16,  8.23it/s]

...

  1%|          | 1/135 [00:00<00:18,  7.08it/s]

Collectting model's outputs


  1%|          | 1/135 [00:00<00:15,  8.69it/s]

Collectting model's outputs


100%|██████████| 135/135 [00:08<00:00, 16.73it/s]


In [7]:
from src.utils.visualize import visualize_predictions

In [8]:
sequences_batch, labels_batch = next(iter(test_dataloader))

visualize_predictions(
    model=ens_bce,
    model_type="ensemble",
    sequences_batch=sequences_batch,
    labels_batch=labels_batch,
    n_pics=10,
    scale=args_config["predictions"]["scale"],
    step=args_config["predictions"]["step"],
    alpha=args_config["predictions"]["alpha"],
    device="cpu",
)

device: cpu
Desired number of pictures is greater than size of the batch provided.


TypeError: 'ModelBeta' object is not callable

In [5]:
res = all_distances_evaluation_pipeline(
    ens_bce,
    test_dataloader,
    distance="wasserstein_1d",
    device="cpu",
    verbose=True,
    window_size_list=args_config["window_size_list"],
    margin_list=[1, 2, 4],
    anchor_window_type_list=args_config["anchor_window_type_list"],
    threshold_list=np.linspace(0, 1, 100),
)

 10%|▉         | 2/21 [00:00<00:01, 13.80it/s]

Collectting model's outputs


  0%|          | 0/100 [00:00<?, ?it/s]

window_size = 1, anchor_window_type = start


  0%|          | 0/100 [00:00<?, ?it/s]

...

Results for best threshold = 0.25252525252525254
AUDC: 0.0, Time to FA: 11.1526, DD: 1.098, F1: 0.975, Cover: 0.9114, Max Cover: 0.9114
Max F1 with margin 1: 0.9444
Max F1 with margin 2: 0.9682
Max F1 with margin 4: 0.9718
window_size = 3, anchor_window_type = start


  0%|          | 0/100 [00:00<?, ?it/s]

Results for best threshold = 0.2828282828282829
AUDC: 0.0, Time to FA: 11.2139, DD: 1.2536, F1: 0.9736, Cover: 0.902, Max Cover: 0.902
Max F1 with margin 1: 0.8843
Max F1 with margin 2: 0.9515
Max F1 with margin 4: 0.9673
window_size = 3, anchor_window_type = prev


100%|██████████| 100/100 [02:26<00:00,  1.47s/it]

Results for best threshold = 0.23232323232323235
AUDC: 0.0, Time to FA: 11.2393, DD: 1.2027, F1: 0.9754, Cover: 0.9074, Max Cover: 0.9074
Max F1 with margin 1: 0.9159
Max F1 with margin 2: 0.9552
Max F1 with margin 4: 0.97


# Bayes BCE HAR

In [3]:
import torch
from src.ensembles.bayes_models import BaseBayesRnn, BayesCPDModel
from src.metrics.evaluation_pipelines import all_distances_evaluation_pipeline
from src.utils.fix_seeds import fix_seeds

SEED = 42
fix_seeds(SEED)

In [4]:
model_type = "seq2seq"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "bce"
args_config["num_workers"] = 2

In [5]:
base_model = BaseBayesRnn(
    input_size=args_config["model"]["input_size"],
    hidden_dim=args_config["model"]["hidden_dim"],
    n_layers=args_config["model"]["n_layers"],
    drop_prob=args_config["model"]["drop_prob"],
    prior_mu=0.0,
    prior_sigma=0.1,
)

In [6]:
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [7]:
bayes_bce = BayesCPDModel(
    args_config, base_model, train_dataset, test_dataset, n_samples=10
)

path = "saved_models/total_bayes/human_activity/total_bayes_coeff_0.2_prior_mu_0.0_prior_sigma_0.1.pth"
bayes_bce.model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [9]:
threshold_number = 100
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

metrics, (max_th_f1_margins_dict, max_f1_margins_dic), _, _ = evaluation_pipeline(
    bayes_bce,
    test_dataloader,
    threshold_list,
    device="cpu",
    model_type="ensemble",
    verbose=True,
    margin_list=[1, 2, 4],
)

  5%|▍         | 1/21 [00:00<00:03,  5.34it/s]

Collectting model's outputs


100%|██████████| 21/21 [00:03<00:00,  6.40it/s]


TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 

In [8]:
res = all_distances_evaluation_pipeline(
    bayes_bce,
    test_dataloader,
    distance="wasserstein_1d",
    device="cpu",
    verbose=True,
    window_size_list=[1, 2, 3],
    anchor_window_type_list=["start", "prev"],
    threshold_list=np.linspace(0, 1, 50),
)

  5%|▍         | 1/21 [00:00<00:03,  5.60it/s]

Collectting model's outputs


  0%|          | 0/50 [00:00<?, ?it/s]

window_size = 1, anchor_window_type = start


  0%|          | 0/50 [00:00<?, ?it/s]

...

Results for best threshold = 0.2040816326530612
AUDC: 0.0, Time to FA: 11.356, DD: 1.4091, F1: 0.965, Cover: 0.897, Max Cover: 0.897
Max F1 with margin 1: 0.8267
Max F1 with margin 2: 0.9595
Max F1 with margin 4: 0.9622
window_size = 3, anchor_window_type = start


  0%|          | 0/50 [00:00<?, ?it/s]

Results for best threshold = 0.3469387755102041
AUDC: 0.0, Time to FA: 11.3583, DD: 2.3859, F1: 0.9411, Cover: 0.8179, Max Cover: 0.8179
Max F1 with margin 1: 0.0809
Max F1 with margin 2: 0.7509
Max F1 with margin 4: 0.9262
window_size = 3, anchor_window_type = prev


100%|██████████| 50/50 [01:09<00:00,  1.39s/it]

Results for best threshold = 0.22448979591836732
AUDC: 0.0, Time to FA: 11.347, DD: 1.9596, F1: 0.954, Cover: 0.8505, Max Cover: 0.8505
Max F1 with margin 1: 0.3064
Max F1 with margin 2: 0.9098
Max F1 with margin 4: 0.9494


# InDiD HAR

In [12]:
model_type = "seq2seq"

experiments_name = "human_activity"

path_to_config = "configs/" + experiments_name + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "indid"
args_config["num_workers"] = 2

# prepare datasets
train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

# load the models
path_to_models_folder = (
    f"saved_models/{args_config['loss_type']}/{experiments_name}/ens_2"
)

ens_indid = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_indid.load_models_list(path_to_models_folder)

In [13]:
threshold_number = 100
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

metrics, (max_th_f1_margins_dict, max_f1_margins_dic), _, _ = evaluation_pipeline(
    ens_indid,
    test_dataloader,
    threshold_list,
    device="cpu",
    model_type="ensemble",
    verbose=True,
    margin_list=[1, 2, 4],
)

 10%|▉         | 2/21 [00:00<00:01, 16.13it/s]

Collectting model's outputs


100%|██████████| 21/21 [00:01<00:00, 17.49it/s]


TN: 0, FP: 1313, FN: 0, TP: 24, DELAY:0.0, FP_DELAY:0.0, COVER: 0.6667726252804786
TN: 169, FP: 51, FN: 20, TP: 1097, DELAY:0.3118923008441925, FP_DELAY:11.222139358520508, COVER: 0.9930929451790812
TN: 169, FP: 47, FN: 20, TP: 1101, DELAY:0.3118923008441925, FP_DELAY:11.225131034851074, COVER: 0.9933565951416841
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.2266263961792, COVER: 0.993450088036224
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.2266263961792, COVER: 0.993450088036224
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.2266263961792, COVER: 0.993450088036224
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.2266263961792, COVER: 0.993450088036224
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.2266263961792, COVER: 0.993450088036224
TN: 170, FP: 45, FN: 20, TP: 1102, DELAY:0.3118923008441925, FP_DELAY:11.236350059509277, COVER: 0.9936875927928804
TN: 